In [1]:
from sympy import symbols, Or, And, to_cnf, satisfiable

In [2]:
import random

def create_wcnf_file(file_path, num_clauses, num_vars, evidence_length):
    # Generate random clauses with varying lengths and random weights as probabilities
    clauses = [[random.choice([-1, 1]) * random.randint(1, num_vars) for _ in range(random.randint(1, 3))] for _ in range(num_clauses)]
    weights = [random.uniform(0, 1) for _ in range(num_clauses)]  # Generate weights as probabilities

    with open(file_path, 'w') as wcnf_file:
        for clause, weight in zip(clauses, weights):
            weight_line = f"{weight:.2f}"  # Format weight as a floating-point number
            clause_line = ' '.join(map(str, clause + [weight_line])) + ' \n'
            wcnf_file.write(clause_line)

    # Generate evidence with variable assignments
    evidence = generate_evidence(num_vars, evidence_length)

    return evidence

def generate_evidence(num_vars, evidence_length):
    var_list = [i * random.choice([-1, 1]) for i in range(1, num_vars + 1)]  # Assign random signs to variables
    evidence = random.sample(var_list, evidence_length)
    return evidence



In [3]:
num_clauses = 10
num_vars = 5
evidence_length = 3

# Create a WCNF file and generate evidence
create_wcnf_file("random.wcnf", num_clauses, num_vars, evidence_length)
evidence = generate_evidence(num_vars, evidence_length)

In [4]:
class inférence:
    lower = 0
    upper = 0
    weights = []
    formulas = []
    strates_weights = []

    def __init__(self, path_file):
        file = open(path_file)
        for line in file:
            information = line.split()
            size = len(information)
            self.weights.append(float(information[-1]))  # Adjusted to take the weight from the last position
            self.formulas.append([int(information[i]) for i in range(size - 1)])  # Adjusted to exclude the last element
        self.length = len(self.weights)
        file.close()

    def get_length(self):
        return self.length

    def get_weights(self):
        return self.weights

    def get_formulas(self):
        return self.formulas

    def sort_weights(self):
        for i in range(len(self.weights) - 1, 0, -1):
            for j in range(i):
                if self.weights[j] < self.weights[j + 1]:
                    self.weights[j], self.weights[j + 1] = self.weights[j + 1], self.weights[j]
                    self.formulas[j], self.formulas[j + 1] = self.formulas[j + 1], self.formulas[j]

    def compute_strates(self):
        no_doubles = set(self.weights)
        self.upper = len(no_doubles)
        self.strates_weights = sorted(no_doubles)

    def get_strates_number(self):
        return self.upper

    def get_strates_weights(self):
        return self.strates_weights

    def get_preprocessed_formulas(self, sub_formulas):
        dict_cor = {}
        returned_formulas = []
        cpt = 1
        i = 0
        for form in sub_formulas:
            j = 0
            returned_formulas.insert(i, [])
            for pred in form:
                if pred not in dict_cor.keys():
                    if int(pred) > 0:
                        dict_cor[int(pred)] = cpt
                        dict_cor[-int(pred)] = -cpt
                        cpt += 1
                    else:
                        dict_cor[-int(pred)] = cpt
                        dict_cor[int(pred)] = -cpt
                        cpt += 1
                returned_formulas[i].insert(j, dict_cor[pred])
                j += 1
            i += 1
        return returned_formulas


In [5]:
# Création et exécution de notre algorithme d'inférence
inférence = inférence("my.wcnf")  
evidence = [1, -2, -3]

inférence.sort_weights()
inférence.compute_strates()
iteration = 1

In [6]:
while inférence.lower < inférence.upper:
    r = int((inférence.lower + inférence.upper + 1) / 2)

    liste = inférence.get_weights()
    value_of_r = -1
    for i in range(len(liste)):
        if inférence.get_strates_weights()[r - 1] > liste[i]:
            value_of_r = i
            break
    if value_of_r == -1:
        value_of_r = len(liste) - 1

    for j in range(len(liste)):
        if inférence.get_strates_weights()[inférence.upper - 1] == liste[j]:
            valueU = j
            break
    cnf = inférence.formulas[valueU:value_of_r]

    for i in range(len(evidence)):
        liste = []
        liste.append(-1 * evidence[i])
        cnf.append(liste)
    cnf = inférence.get_preprocessed_formulas(cnf)

    symbols_list = symbols([str(abs(lit)) for clause in cnf for lit in clause])
    cnf_expr = And(
        *[Or(*[symbols_list[abs(lit) - 1] if lit > 0 else ~symbols_list[abs(lit) - 1] for lit in clause]) for clause in
          cnf])
    result = satisfiable(to_cnf(cnf_expr))

    if result:
        inférence.upper = r - 1
    else:
        inférence.lower = r

    iteration = iteration + 1

Val = inférence.get_strates_weights()[r - 1]
print("Evidence is ", evidence)
print("Result of Pi: Val(evidence, Sigma) = " + str(Val))


Evidence is  [1, -2, -3]
Result of Pi: Val(evidence, Sigma) = 0.5
